In [16]:
# Importing Libraries

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import fancyimpute as fi
import warnings
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
warnings.filterwarnings('ignore')

In [81]:
# Loading Data

train_df = pd.DataFrame.from_csv('train.csv')
test_df = pd.DataFrame.from_csv('test.csv')

In [82]:
train_df.head(10)

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
Loan_ID,,,,,,,,,,,,
LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
LP001011,Male,Yes,2,Graduate,Yes,5417,4196.0,267.0,360.0,1.0,Urban,Y
LP001013,Male,Yes,0,Not Graduate,No,2333,1516.0,95.0,360.0,1.0,Urban,Y
LP001014,Male,Yes,3+,Graduate,No,3036,2504.0,158.0,360.0,0.0,Semiurban,N
LP001018,Male,Yes,2,Graduate,No,4006,1526.0,168.0,360.0,1.0,Urban,Y


Dealing with Missing Values

In [60]:
# Detection
missing_vals = pd.concat([train_df.isnull().sum() * 100/len(train_df),test_df.isnull().sum() * 100/len(test_df)],axis=1, keys=['Train','Test'])
missing_vals.sort_values(ascending=False,by="Train")

,Train,Test
Credit_History,8.143322,7.901907
Self_Employed,5.211726,6.267030
LoanAmount,3.583062,1.362398
Dependents,2.442997,2.724796
Loan_Amount_Term,2.280130,1.634877
term,2.280130,NaN
Gender,2.117264,2.997275
Married,0.488599,0.000000
ApplicantIncome,0.000000,0.000000
CoapplicantIncome,0.000000,0.000000


In [83]:
# Basic Methods
base_imp_train = train_df.copy()
mode_columns = ['Credit_History','Self_Employed','Dependents','Loan_Amount_Term','Gender','Married']
median_cols = ['LoanAmount']

for i in mode_columns:
    base_imp_train[i] = base_imp_train[i].fillna(base_imp_train[i].mode()[0])

base_imp_train['LoanAmount'] = base_imp_train['LoanAmount'].fillna(base_imp_train['LoanAmount'].median())


Creating Dummies

In [84]:
# Dropping first column created by dummy
imp_train_dum = pd.get_dummies(base_imp_train,columns = ['Gender','Married','Dependents','Education','Self_Employed','Property_Area'],drop_first = True)
imp_train_dum['Loan_Status'] = np.where(imp_train_dum['Loan_Status'] == 'Y',1,0)


In [72]:
# imp_train_dum['total_income'] =  imp_train_dum['ApplicantIncome'] + imp_train_dum['CoapplicantIncome']

In [85]:
# Train Test Split

X = imp_train_dum.drop(['Loan_Status','Loan_Amount_Term'],axis = 1)
y = pd.DataFrame(imp_train_dum['Loan_Status'])
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)



In [86]:
clf = LogisticRegression(penalty='l1',C=1)
# clf = RandomForestClassifier()
clf.fit(X_train,y_train)
# clf.score(X_test,y_test)

LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l1', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [87]:
clf.score(X_test,y_test)

0.8373983739837398

In [88]:

for i in mode_columns:
    test_df[i] = test_df[i].fillna(test_df[i].mode()[0])

test_df['LoanAmount'] = test_df['LoanAmount'].fillna(test_df['LoanAmount'].median())

In [91]:
test_df = pd.get_dummies(test_df,columns = ['Gender','Married','Dependents','Education','Self_Employed','Property_Area'],drop_first = True)


KeyError: "['Gender' 'Married' 'Dependents' 'Education' 'Self_Employed'\n 'Property_Area'] not in index"

In [93]:
test_df = test_df.drop(['Loan_Amount_Term'],axis = 1)

In [94]:
sub_df = pd.DataFrame()
sub_df['Loan_ID'] = test_df.index
sub_df['Loan_Status'] = clf.predict(test_df)

In [95]:
sub_df['Loan_Status'] = np.where(sub_df['Loan_Status']==1,'Y','N')
sub_df.to_csv('sub2.csv',index = False)